In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, regularizers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import os
import zipfile
from sklearn.utils import class_weight
from skimage.feature import local_binary_pattern

In [2]:
# Define paths for the zip file and its extraction location
zip_path = '/content/faces_224.zip'
unzip_path = '/content'
# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)
# Update the image path
image_path = unzip_path

In [3]:
def create_efficientnet_model(input_shape=(224, 224, 3), feature_shape=(262,)):
    """
    Create model with properly separated image and feature inputs
    """
    # Image input branch
    image_input = layers.Input(shape=input_shape, name="image_input")

    # Create EfficientNetB0 as a separate model first
    efficient_net = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    efficient_net.trainable = False

    # Process image through EfficientNet
    x = efficient_net(image_input)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    # Handcrafted feature input branch (separate from EfficientNet)
    feature_input = layers.Input(shape=feature_shape, name="feature_input")

    # Process handcrafted features
    y = layers.BatchNormalization()(feature_input)
    y = layers.Dense(256, activation='relu')(y)
    y = layers.Dropout(0.3)(y)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.Dropout(0.2)(y)

    # Combine features
    combined = layers.Concatenate()([x, y])

    # Final classification layers
    z = layers.Dense(256, activation='relu')(combined)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.5)(z)
    z = layers.Dense(128, activation='relu')(z)
    z = layers.BatchNormalization()(z)
    z = layers.Dropout(0.3)(z)
    outputs = layers.Dense(1, activation='sigmoid')(z)

    # Create model with both inputs
    model = Model(
        inputs={
            "image_input": image_input,
            "feature_input": feature_input
        },
        outputs=outputs
    )

    return model


In [4]:
class DeepFakeDataGenerator:
    def __init__(self, dataframe, image_path, batch_size=32, target_size=(224, 224)):
        self.dataframe = dataframe
        self.image_path = image_path
        self.batch_size = batch_size
        self.target_size = target_size
        self.n = len(dataframe)
        self.indexes = np.arange(self.n)

    def __len__(self):
        return int(np.ceil(self.n / self.batch_size))

    def on_epoch_end(self):
        np.random.shuffle(self.indexes)

    def __call__(self):
        while True:
            for i in range(0, self.n, self.batch_size):
                batch_indexes = self.indexes[i:min(i + self.batch_size, self.n)]
                batch_data = self.dataframe.iloc[batch_indexes]

                images = []
                features = []
                labels = []

                for _, row in batch_data.iterrows():
                    img_path = os.path.join(self.image_path, row['videoname'][:-4] + '.jpg')
                    try:
                        image = cv2.imread(img_path)
                        if image is None:
                            continue

                        image = cv2.resize(image, self.target_size)
                        handcrafted_features = extract_features(image)
                        image_normalized = image.astype('float32') / 255.0

                        images.append(image_normalized)
                        features.append(handcrafted_features)
                        labels.append(1 if row['label'] == 'FAKE' else 0)

                    except Exception as e:
                        print(f"Error processing {img_path}: {str(e)}")
                        continue

                if not images:
                    continue

                images = np.array(images, dtype=np.float32)
                features = np.array(features, dtype=np.float32)
                labels = np.array(labels)

                yield {"image_input": images, "feature_input": features}, labels

In [10]:
def train_model(train_gen, val_gen, steps_per_epoch, validation_steps, epochs=20):
    """
    Create and train the model
    """
    model = create_efficientnet_model(input_shape=(224, 224, 3), feature_shape=(262,))

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-6
        )
    ]

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        epochs=epochs,
        callbacks=callbacks,
        verbose =1
    )

    return model, history

def fine_tune_model(model, train_gen, val_gen, steps_per_epoch, validation_steps, epochs=10):
    """
    Fine-tune the model
    """
    # Find the EfficientNet model
    efficient_net = None
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model):
            efficient_net = layer
            break

    if efficient_net is not None:
        efficient_net.trainable = True

        # Freeze first 70% of the layers
        num_layers = len(efficient_net.layers)
        num_to_freeze = int(0.7 * num_layers)
        for layer in efficient_net.layers[:num_to_freeze]:
            layer.trainable = False

    # Recompile with lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=3,
                restore_best_weights=True
            )
        ],
        verbose=1
    )

    return history


In [6]:
def extract_features(image):
    """
    Extract image features with exactly 262 dimensions
    """
    if image.dtype == np.float32 and image.max() <= 1.0:
        image = (image * 255).astype(np.uint8)
    elif image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Color histogram (216 features)
    hist = cv2.calcHist([image], [0, 1, 2], None, [6, 6, 6], [0, 256, 0, 256, 0, 256])
    color_hist = cv2.normalize(hist, hist).flatten()

    # Edge features (16 features)
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    gray = gray.astype(np.uint8)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 100, 200)
    edge_hist = cv2.calcHist([edges], [0], None, [16], [0, 256])
    edge_hist = cv2.normalize(edge_hist, edge_hist).flatten()

    # LBP features (10 features)
    lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), range=(0, 10))
    lbp_hist = lbp_hist.astype(np.float32)
    lbp_hist = lbp_hist / (lbp_hist.sum() + 1e-7)

    # Statistical features (20 features)
    stat_features = []
    # Global statistics
    stat_features.extend([
        np.mean(gray),
        np.std(gray),
        np.median(gray),
        np.percentile(gray, 25),
        np.percentile(gray, 75),
    ])

    # Region-based statistics (3 regions: top, middle, bottom)
    h = gray.shape[0]
    regions = [
        gray[:h//3],           # top
        gray[h//3:2*h//3],     # middle
        gray[2*h//3:]          # bottom
    ]

    for region in regions:
        stat_features.extend([
            np.mean(region),
            np.std(region),
            np.median(region),
            np.percentile(region, 25),
            np.percentile(region, 75),
        ])

    stat_features = np.array(stat_features, dtype=np.float32)

    # Total features: 216 (color) + 16 (edge) + 10 (LBP) + 20 (statistical) = 262 features
    features = np.concatenate([color_hist, edge_hist, lbp_hist, stat_features])
    return features


In [ ]:
if __name__ == "__main__":
    # Load metadata
    metadata_path = "/content/metadata.csv"
    df = pd.read_csv(metadata_path)

    # Sample and split data
    real_df = df[df["label"] == "REAL"]
    fake_df = df[df["label"] == "FAKE"]
    sample_size = 10000
    real_df = real_df.sample(sample_size, random_state=42)
    fake_df = fake_df.sample(sample_size, random_state=42)
    sample_meta = pd.concat([real_df, fake_df])

    # Split into train, validation, and test sets
    Train_set, Test_set = train_test_split(sample_meta, test_size=0.2, random_state=42, stratify=sample_meta['label'])
    Train_set, Val_set = train_test_split(Train_set, test_size=0.3, random_state=42, stratify=Train_set['label'])

    # Create data generators
    batch_size = 32
    train_generator = DeepFakeDataGenerator(Train_set, image_path, batch_size)
    val_generator = DeepFakeDataGenerator(Val_set, image_path, batch_size)
    test_generator = DeepFakeDataGenerator(Test_set, image_path, batch_size)

    # Calculate steps per epoch
    steps_per_epoch = len(train_generator)
    validation_steps = len(val_generator)

    # Create and train the initial model
    model = create_efficientnet_model(input_shape=(224, 224, 3), feature_shape=(288,))

    # Initial training
    history = train_model(
        train_generator(),
        val_generator(),
        steps_per_epoch,
        validation_steps,
        epochs=20,
    )

    # Fine-tuning
    # Unfreeze the EfficientNet layers
    base_model = model.get_layer('efficientnetb0')
    base_model.trainable = True

    # Recompile with a lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Fine-tune the model
    history_ft = model.fit(
        train_generator(),
        steps_per_epoch=steps_per_epoch,
        validation_data=val_generator(),
        validation_steps=validation_steps,
        epochs=10,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        ],
    )

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(
        test_generator(),
        steps=len(test_generator)
    )
    print(f"Final Test Accuracy: {test_accuracy:.4f}")
    print(f"Final Test Loss: {test_loss:.4f}")

    # Save the model
    model.save('deepfake_detector_efficientnetb0.h5')

    # If running in Google Colab
    try:
        from google.colab import files
        files.download('deepfake_detector_efficientnetb0.h5')
    except ImportError:
        print("Model saved locally as 'deepfake_detector_efficientnetb0.h5'")

    # Create visualization of training history
    plt.figure(figsize=(15, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    # Initial training
    plt.plot(history[0].history['accuracy'], label='Training Accuracy')
    plt.plot(history[0].history['val_accuracy'], label='Validation Accuracy')
    # Fine-tuning
    if len(history_ft.history['accuracy']) > 0:
        offset = len(history[0].history['accuracy'])
        epochs_ft = np.arange(offset, offset + len(history_ft.history['accuracy']))
        plt.plot(epochs_ft, history_ft.history['accuracy'],
                label='Fine-tuning Training Accuracy', linestyle='--')
        plt.plot(epochs_ft, history_ft.history['val_accuracy'],
                label='Fine-tuning Validation Accuracy', linestyle='--')

    plt.title('Model Accuracy Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

    # Plot loss
    plt.subplot(1, 2, 2)
    # Initial training
    plt.plot(history[0].history['loss'], label='Training Loss')
    plt.plot(history[0].history['val_loss'], label='Validation Loss')
    # Fine-tuning
    if len(history_ft.history['loss']) > 0:
        offset = len(history[0].history['loss'])
        epochs_ft = np.arange(offset, offset + len(history_ft.history['loss']))
        plt.plot(epochs_ft, history_ft.history['loss'],
                label='Fine-tuning Training Loss', linestyle='--')
        plt.plot(epochs_ft, history_ft.history['val_loss'],
                label='Fine-tuning Validation Loss', linestyle='--')

    plt.title('Model Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Print final metrics summary
    print("\nTraining Summary:")
    print(f"Best validation accuracy: {max(history[0].history['val_accuracy']):.4f}")
    print(f"Best validation loss: {min(history[0].history['val_loss']):.4f}")
    if len(history_ft.history['val_accuracy']) > 0:
        print(f"Best fine-tuning validation accuracy: {max(history_ft.history['val_accuracy']):.4f}")
        print(f"Best fine-tuning validation loss: {min(history_ft.history['val_loss']):.4f}")
    print(f"Final test accuracy: {test_accuracy:.4f}")
    print(f"Final test loss: {test_loss:.4f}")